## 4. Once again, consider the MNIST dataset. Use the training set as your training data and the test set as your test data.

In [1]:
import gzip
import time
import numpy as np
import matplotlib.pyplot as plt

image_size = 28
training_samples = 60000

f_train = gzip.open('train-images-idx3-ubyte.gz','r')
f_train.read(16)
buf = f_train.read(image_size * image_size * training_samples)
train_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
# train_data = data.reshape(num_images, image_size, image_size)
train_data = train_data.reshape(training_samples, image_size, image_size)


f_train_label = gzip.open('train-labels-idx1-ubyte.gz','r')
f_train_label.read(8)
buf = f_train_label.read(training_samples)
train_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)


testing_images = 10000
f_test = gzip.open('t10k-images-idx3-ubyte.gz','r')
f_test.read(16)
buf = f_test.read(image_size * image_size * testing_images)
test_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
test_data = test_data.reshape(testing_images, image_size , image_size)


f_test_label = gzip.open('t10k-labels-idx1-ubyte.gz','r')
f_test_label.read(8)
buf = f_test_label.read(testing_images)
test_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)


# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# print(train_labels[10])
# print(train_data[10])

a) Classify the test data using a kNN classifier. Report the accuracy for k = {1,3,5,11}. Justify and compare the reported accuracies for the different values of k. Do not use kNN implemented function in Python/Matlab and implement it by yourself.

In [2]:
from scipy.spatial.distance import cdist

def get_euclidean_distance(sample1, sample2):
    return cdist(sample2, sample1 , 'euclidean')


def my_knn(train_data, train_labels, test_data, k_for_knn):
    
    distance_matrix = get_euclidean_distance(train_data, test_data)
    distance_matrix=np.array(distance_matrix)
    distance_matrix_sorted_index=np.argsort(distance_matrix)
    
    accuracies = []
    for k in k_for_knn:
    
        correct_classified = 0
        for i in range(len(distance_matrix_sorted_index)):

            array = distance_matrix_sorted_index[i][:k]

            neighbours_info = []        
            for i2 in array:
                if len(neighbours_info) == 0:
                    neighbours_info.append([train_labels[i2],1])
                else:
                    index = 0
                    while index < len(neighbours_info):
                        if neighbours_info[index][0] == train_labels[i2]: 
                            break 
                        index+=1

                    if index < len(neighbours_info):
                        neighbours_info[index][1]+=1
                    else:    
                        neighbours_info.append([train_labels[i2],1]) 
            neighbours_info=np.array(neighbours_info) 
            neighbours_info = neighbours_info[neighbours_info[:,1].argsort()]
            neighbours_info=neighbours_info[::-1]

            if test_labels[i] == neighbours_info[0][0]:
                correct_classified+=1

        accuracy = (correct_classified/len(distance_matrix_sorted_index))*100
        accuracies.append(accuracy)
    
    return accuracies

In [3]:
my_train_data = train_data.reshape(training_samples, image_size * image_size)
my_test_data = test_data.reshape(testing_images, image_size * image_size)
k= [1,3,5,11]

t1 = time.time()
accuracies = my_knn(my_train_data, train_labels, my_test_data, k)
t2 = time.time()
print("Completed!")
print("k=1 :", accuracies[0])
print("k=3 :", accuracies[1])
print("k=5 :", accuracies[2])
print("k=11 :", accuracies[3])
print("Time Spent:",str(t2-t1)[:5]," seconds.")

Completed!
k=1 : 96.91
k=3 : 97.04
k=5 : 96.86
k=11 : 96.64
Time Spent: 886.3  seconds.


b) Apply PCA to MNIST to create a new dataset MNIST-d. Classify the test samples in MNIST-d using a kNN classifier. For each d = {5, 50, 100, 500} use k = {1,3,5,11}. Calculate and display the classification accuracy for each of the 16 combinations of d and k in a table. Discuss the results.

In [5]:
def pca_from_no_dimentions(data, dimentions):
    X = data
    mean_train_data = np.mean(X, axis=0)
    X = X - mean_train_data
    cov = np.cov(X)
    eigen_values, eigen_vectors = np.linalg.eigh(cov)

    i = np.argsort(eigen_values)
    i=i[::-1]
    largest_eigen_values_index = i[:dimentions]
    eigen_vectors_with_largest_eigen_values = eigen_vectors[:,largest_eigen_values_index]
    P = np.dot(eigen_vectors_with_largest_eigen_values.T,X) 
    return np.array(P), eigen_vectors_with_largest_eigen_values, eigen_values, mean_train_data

In [7]:
my_train_data = train_data.reshape(training_samples, image_size * image_size)
my_test_data = test_data.reshape(testing_images, image_size * image_size)
d=[5,50,100,500]
k=[1,3,5,11]

t1 = time.time()
for val in d:
    print("\nExecuting for d =",val,"")
    t1 = time.time()
    lower_dimentions_data_train, _, _, _ = pca_from_no_dimentions(np.array(my_train_data.T), val)
    lower_dimentions_data_test, _, _, _ = pca_from_no_dimentions(np.array(my_test_data.T), val)
    accuracies = my_knn(lower_dimentions_data_train.T, train_labels, lower_dimentions_data_test.T, k)
    print("For d =",val," k=1 :", accuracies[0]," k=3 :", accuracies[1]," k=5 :", accuracies[2]," k=11 :", accuracies[3])
t2 = time.time()
print("\nCompleted!","Time Spent:",str(t2-t1)[:5]," seconds.")    


Executing for d = 5 
For d = 5  k=1 : 37.2  k=3 : 39.01  k=5 : 40.03  k=11 : 40.82

Executing for d = 50 
For d = 50  k=1 : 41.47  k=3 : 43.69  k=5 : 44.7  k=11 : 46.26

Executing for d = 100 
For d = 100  k=1 : 42.52  k=3 : 44.49  k=5 : 45.35  k=11 : 46.0

Executing for d = 500 
For d = 500  k=1 : 43.16  k=3 : 44.39  k=5 : 45.12  k=11 : 45.9

Completed! Time Spent: 656.9  seconds.
